https://docs.langchain.com/oss/python/langchain/knowledge-base

##Importante

Percebi que com textos em portguês não foi possível localizar o conteúdo por nenhum dos métodos do langchain-core. preciso entender se o que aconteceu. Criei outro arquivo com pdf em inglês e perguntas também e inglês e funcionou bem.

In [ ]:
pip install langchain-community pypdf langchain-openai ipywidgets langchain-core langchain-openai faiss-cpu zod -q

##talvez substiruir por:
# pip install langchain langchain-openai faiss-cpu pypdf ipywidgets -q


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import os

# apikeyGpt = os.environ.get('LANGSMITH_API_KEY')
# if not apikeyGpt:
#     raise RuntimeError("LANGSMITH_API_KEY not set. Set it in PowerShell or Windows Environment Variables.")

# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = apikeyGpt


In [45]:
## Loading documents
from langchain_community.document_loaders import PyPDFLoader

file_path = "./arquivos/manual_avancado_nobreak_fxp2000.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

print('Paginas:' +  str(len(docs)))
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)


Paginas:3
Manual Técnico - Nobreak FXP-2000
1. Introdução
O Nobreak FXP-2000 foi desenvolvido para garantir proteção e autonomia a equipamentos eletrônicos
sensíveis. Ideal para ambientes domésticos e corporati

{'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250412132946', 'source': './arquivos/manual_avancado_nobreak_fxp2000.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}


In [46]:
##Splitting documents into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

# print(len(all_splits))

# print(f"{all_splits[3].page_content}")

In [47]:
apikeyGpt = os.environ.get('GPT_APIKEY')
if not apikeyGpt:
    raise RuntimeError("GPT_APIKEY not set. Set it in PowerShell or Windows Environment Variables.")

os.environ["OPENAI_API_KEY"] = apikeyGpt

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [48]:
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

chat = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
def simple_query(query, k=3):
    """Recupera top-k documentos, junta um contexto curto e chama o chat."""
    # 1. recuperar documentos
    try:
        docs = vectorstore.similarity_search(query, k)
    except Exception:
        docs = vectorstore.similarity_search(query)

    if not docs:
        print('Nenhum documento encontrado para a pergunta.')
        return None

    # 2. mostrar pequenos trechos
    print(f'Retrieved {len(docs)} docs (top {k}):')
    for i, d in enumerate(docs):
        text = getattr(d, 'page_content', getattr(d, 'content', str(d)))
        print('--- doc', i, 'len=', len(text))
        print(text[:400].replace('\n', ' '))

    # 3. montar contexto curto
    context = '\n\n---\n\n'.join(getattr(d, 'page_content', getattr(d, 'content', str(d))) for d in docs)
    if len(context) > 3000:
        context = context[:10000] + '\n\n... (truncated)'

    # 4. preparar prompt
    prompt = (f'Use o contexto abaixo para responder em Português.\n\nContext:\n{context}\n\nPergunta: {query}\n\nResposta sucinta:')

    print(f'Pergunta: {prompt}')

    # 5. chamar o chat (tenta predict, depois messages)
    try:
        answer = chat.invoke(prompt)
        print('--- Resposta ---')
        print(answer.content)
        return answer.content
    except Exception:
        try:
            from langchain_core.messages import SystemMessage, HumanMessage
            messages = [SystemMessage(content='Você é um assistente útil.'), HumanMessage(content=prompt)]
            answer = chat(messages)
            print('--- Resposta ---')
            print(answer.content)
            return answer.content
        except Exception as e:
            print('Falha ao chamar o modelo. Verifique `chat` e credenciais:', e)
            return None



In [60]:
# Exemplo: trocar a pergunta e executar
res = simple_query('Qual a eficiência de autonomia do nobreak FXP2000 com 50% de carga?', k=3)

Retrieved 3 docs (top 3):
--- doc 0 len= 1207
Manual Técnico - Nobreak FXP-2000 - Porta USB Tipo A e Tipo C para carregamento de dispositivos 6. Modos de Operação - Modo Rede: operação normal com carga alimentada pela rede - Modo Bateria: ativado automaticamente durante falha da rede elétrica - Modo Economia (Battery Saver): desativa a saída caso nenhuma carga seja detectada - Modo Emergência: iluminação LED branca acionada durante falha elét
--- doc 1 len= 1439
Manual Técnico - Nobreak FXP-2000 1. Introdução O Nobreak FXP-2000 foi desenvolvido para garantir proteção e autonomia a equipamentos eletrônicos sensíveis. Ideal para ambientes domésticos e corporativos, seu sistema inteligente de gerenciamento de energia assegura operação estável mesmo em condições adversas da rede elétrica. 2. Aplicações Recomendadas Equipamentos compatíveis incluem: computador
--- doc 2 len= 244
Manual Técnico - Nobreak FXP-2000 10. Suporte Técnico Em caso de dúvidas ou necessidade de manutenção, acesse nos